In [17]:
import os
print(os.getcwd())

/Users/liameliot/Documents/DA 401/CFB


In [50]:
#Import packages and Functions
import numpy as np
from scipy.stats import norm
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import random
%run Functions.ipynb



/Users/liameliot/Documents/DA 401/CFB


## 2024 Season

In [55]:
eloranks = pnl

# Sets Tiers

tiers_dict = tiers(eloranks)
tier1 = tiers_dict["tier1"]
tier2 = tiers_dict["tier2"]
tier3 = tiers_dict["tier3"]
tier4 = tiers_dict["tier4"]


# Runs Season Sim

# tier 1 results
final_results, matchup_df = simulate_season(tier1)
# tier 2 results
final_results2, matchup_df2 = simulate_season(tier2)
# tier 3 results
final_results3, matchup_df3 = simulate_season(tier3)
# tier 4 results
final_results4, matchup_df4 = simulate_season(tier4)


final_results['Tier'] = "Tier 1"
final_results2['Tier'] = "Tier 2"
final_results3['Tier'] = "Tier 3"
final_results4['Tier'] = "Tier 4"

Tier_1_CBR_24 = matchup_df['CBR'].mean()
Tier_2_CBR_24 = matchup_df2['CBR'].mean()
Tier_3_CBR_24 = matchup_df3['CBR'].mean()
Tier_4_CBR_24 = matchup_df4['CBR'].mean()

# Final Season Results

cfb24 = pd.concat([final_results, final_results2, final_results3, final_results4], ignore_index=True)
cfb24['Year'] = 2024
cfb24



,Team,Wins,Losses,Starting ELO,Week 12 ELO,Starting Rank,Week 12 Rank,Home Games,Away Games,Tier,Year
0,Michigan,9,3,1743.400000,2003.0,2,1,6,6,Tier 1,2024
1,Tulane,9,3,1621.000000,1989.0,16,2,4,8,Tier 1,2024
2,LSU,8,4,1678.766667,1951.0,8,3,7,5,Tier 1,2024
3,Iowa,8,4,1608.900000,1864.0,19,6,6,6,Tier 1,2024
4,Texas A&M,8,4,1580.866667,1837.0,30,8,4,8,Tier 1,2024
...,...,...,...,...,...,...,...,...,...,...,...
129,Temple,5,7,1364.866667,1365.0,28,25,7,5,Tier 4,2024
130,Central Michigan,3,9,1431.000000,1258.0,11,30,6,6,Tier 4,2024
131,UMass,5,7,1330.500000,1331.0,30,28,6,6,Tier 4,2024
132,Colorado State,2,10,1433.900000,1178.0,9,32,7,5,Tier 4,2024


## Re Rank Season 2

In [67]:
# Season 2
preseason25 = sortelo(cfb24)
preseason25['ELO'] = preseason25['ELO'] * 0.95
eloranks = preseason25
eloranks

,Team,ELO,Home Games,Away Games,Year
0,Michigan,1902.85,6,6,2024
1,Tulane,1889.55,4,8,2024
2,LSU,1853.45,7,5,2024
3,Notre Dame,1820.20,6,6,2024
4,Arizona State,1804.05,3,9,2024
...,...,...,...,...,...
129,Bowling Green,1235.95,5,7,2024
130,Central Michigan,1195.10,6,6,2024
131,North Texas,1178.95,7,5,2024
132,Louisiana Tech,1154.25,5,7,2024


## 2025 Season

In [68]:
# Sets Tiers

tiers_dict = tiers(eloranks)
tier1 = tiers_dict["tier1"]
tier2 = tiers_dict["tier2"]
tier3 = tiers_dict["tier3"]
tier4 = tiers_dict["tier4"]

# Runs Season Sim

# tier 1 results
final_results, matchup_df = simulate_season(tier1)
# tier 2 results
final_results2, matchup_df2 = simulate_season(tier2)
# tier 3 results
final_results3, matchup_df3 = simulate_season(tier3)
# tier 4 results
final_results4, matchup_df4 = simulate_season(tier4)


final_results['Tier'] = "Tier 1"
final_results2['Tier'] = "Tier 2"
final_results3['Tier'] = "Tier 3"
final_results4['Tier'] = "Tier 4"

Tier_1_CBR_25 = matchup_df['CBR'].mean()
Tier_2_CBR_25 = matchup_df2['CBR'].mean()
Tier_3_CBR_25 = matchup_df3['CBR'].mean()
Tier_4_CBR_25 = matchup_df4['CBR'].mean()


# Final Season Results

cfb25 = pd.concat([final_results, final_results2, final_results3, final_results4], ignore_index=True)
cfb25['Year'] = 2025
cfb25

,Team,Wins,Losses,Starting ELO,Week 12 ELO,Starting Rank,Week 12 Rank,Home Games,Away Games,Tier,Year
0,Notre Dame,9,3,1820.20,2153.0,4,1,6,6,Tier 1,2025
1,Georgia Tech,8,4,1759.40,2025.0,11,4,7,5,Tier 1,2025
2,Michigan,6,6,1902.85,1999.0,1,5,7,5,Tier 1,2025
3,LSU,8,4,1853.45,2055.0,3,2,7,5,Tier 1,2025
4,Indiana,9,3,1706.20,2040.0,17,3,6,6,Tier 1,2025
...,...,...,...,...,...,...,...,...,...,...,...
129,Connecticut,5,7,1287.25,1271.0,22,25,4,8,Tier 4,2025
130,Texas State,3,9,1339.50,1154.0,12,30,6,6,Tier 4,2025
131,Colorado State,6,6,1119.10,1210.0,32,28,7,5,Tier 4,2025
132,UCLA,2,10,1355.65,1029.0,10,32,6,6,Tier 4,2025


## Re Rank Season 3

In [25]:
# Season 3
preseason26 = sortelo(cfb25)
preseason26['ELO'] = preseason26['ELO'] * 0.95
eloranks = preseason26
eloranks

,Team,ELO,Home Games,Away Games,Year
0,Michigan,2755.95,4,8,2025
1,South Alabama,2708.45,3,9,2025
2,Washington,2699.90,3,9,2025
3,Wisconsin,2677.10,3,9,2025
4,Kansas State,2662.85,6,6,2025
...,...,...,...,...,...
129,Appalachian State,1907.60,8,4,2025
130,Houston,1897.15,6,6,2025
131,Arkansas,1867.70,7,5,2025
132,Coastal Carolina,1857.25,11,1,2025


## 2026 Season

In [26]:
# Sets Tiers

tiers_dict = tiers(eloranks)
tier1 = tiers_dict["tier1"]
tier2 = tiers_dict["tier2"]
tier3 = tiers_dict["tier3"]
tier4 = tiers_dict["tier4"]

# Runs Season Sim

# tier 1 results
final_results, matchup_df = simulate_season(tier1)
# tier 2 results
final_results2, matchup_df2 = simulate_season(tier2)
# tier 3 results
final_results3, matchup_df3 = simulate_season(tier3)
# tier 4 results
final_results4, matchup_df4 = simulate_season(tier4)


final_results['Tier'] = "Tier 1"
final_results2['Tier'] = "Tier 2"
final_results3['Tier'] = "Tier 3"
final_results4['Tier'] = "Tier 4"

Tier_1_CBR_26 = matchup_df['CBR'].mean()
Tier_2_CBR_26 = matchup_df2['CBR'].mean()
Tier_3_CBR_26 = matchup_df3['CBR'].mean()
Tier_4_CBR_26 = matchup_df4['CBR'].mean()


# Final Season Results

cfb26 = pd.concat([final_results, final_results2, final_results3, final_results4], ignore_index=True)
cfb26['Year'] = 2026
cfb26

,Team,Wins,Losses,Starting ELO,Week 12 ELO,Starting Rank,Week 12 Rank,Home Games,Away Games,Tier,Year
0,Wyoming,8,4,2612.50,3292.0,8,3,4,8,Tier 1,2026
1,Kansas State,8,4,2662.85,3343.0,5,1,4,8,Tier 1,2026
2,Minnesota,8,4,2567.85,3248.0,14,6,4,8,Tier 1,2026
3,Notre Dame,9,3,2535.55,3301.0,18,2,3,9,Tier 1,2026
4,Ohio,8,4,2512.75,3193.0,21,8,4,8,Tier 1,2026
...,...,...,...,...,...,...,...,...,...,...,...
129,Appalachian State,5,7,1907.60,2333.0,28,30,7,5,Tier 4,2026
130,Arkansas,6,6,1867.70,2378.0,30,28,6,6,Tier 4,2026
131,Kent State,6,6,1781.25,2291.0,32,31,6,6,Tier 4,2026
132,Coastal Carolina,6,6,1857.25,2367.0,31,29,6,6,Tier 4,2026


## Re rank season 4

In [27]:
# Season 4
preseason27 = sortelo(cfb26)
preseason27['ELO'] = preseason27['ELO'] * 0.95
eloranks = preseason27
eloranks

,Team,ELO,Home Games,Away Games,Year
0,Kansas State,3175.85,4,8,2026
1,Notre Dame,3135.95,3,9,2026
2,Wyoming,3127.40,4,8,2026
3,Toledo,3104.60,3,9,2026
4,Michigan,3102.70,6,6,2026
...,...,...,...,...,...
129,Coastal Carolina,2248.65,6,6,2026
130,California,2244.85,10,2,2026
131,Appalachian State,2216.35,7,5,2026
132,Kent State,2176.45,6,6,2026


## 2027 Season

In [28]:
# Sets Tiers

tiers_dict = tiers(eloranks)
tier1 = tiers_dict["tier1"]
tier2 = tiers_dict["tier2"]
tier3 = tiers_dict["tier3"]
tier4 = tiers_dict["tier4"]

# Runs Season Sim

# tier 1 results
final_results, matchup_df = simulate_season(tier1)
# tier 2 results
final_results2, matchup_df2 = simulate_season(tier2)
# tier 3 results
final_results3, matchup_df3 = simulate_season(tier3)
# tier 4 results
final_results4, matchup_df4 = simulate_season(tier4)


final_results['Tier'] = "Tier 1"
final_results2['Tier'] = "Tier 2"
final_results3['Tier'] = "Tier 3"
final_results4['Tier'] = "Tier 4"

Tier_1_CBR_27 = matchup_df['CBR'].mean()
Tier_2_CBR_27 = matchup_df2['CBR'].mean()
Tier_3_CBR_27 = matchup_df3['CBR'].mean()
Tier_4_CBR_27 = matchup_df4['CBR'].mean()


# Final Season Results

cfb27 = pd.concat([final_results, final_results2, final_results3, final_results4], ignore_index=True)
cfb27['Year'] = 2027
cfb27

,Team,Wins,Losses,Starting ELO,Week 12 ELO,Starting Rank,Week 12 Rank,Home Games,Away Games,Tier,Year
0,Wisconsin,9,3,2946.90,3712.0,16,3,3,9,Tier 1,2027
1,Tulane,9,3,2994.40,3759.0,10,1,3,9,Tier 1,2027
2,SMU,9,3,2904.15,3669.0,21,5,3,9,Tier 1,2027
3,Ohio,8,4,3033.35,3713.0,8,2,4,8,Tier 1,2027
4,Toledo,6,6,3104.60,3615.0,4,7,6,6,Tier 1,2027
...,...,...,...,...,...,...,...,...,...,...,...
129,Georgia Southern,4,8,2342.70,2683.0,23,29,8,4,Tier 4,2027
130,Appalachian State,6,6,2216.35,2726.0,30,27,6,6,Tier 4,2027
131,Kent State,5,7,2176.45,2601.0,31,31,7,5,Tier 4,2027
132,California,5,7,2244.85,2670.0,29,30,7,5,Tier 4,2027


## Re Rank Season 5

In [29]:
# Season 5
preseason28 = sortelo(cfb27)
preseason28['ELO'] = preseason28['ELO'] * 0.95
eloranks = preseason28
eloranks

,Team,ELO,Home Games,Away Games,Year
0,Tulane,3571.05,3,9,2027
1,Ohio,3527.35,4,8,2027
2,Wisconsin,3526.40,3,9,2027
3,Kansas State,3501.70,6,6,2027
4,SMU,3485.55,3,9,2027
...,...,...,...,...,...
129,Oklahoma,2551.70,11,1,2027
130,Georgia Southern,2548.85,8,4,2027
131,California,2536.50,7,5,2027
132,Kent State,2470.95,7,5,2027


## 2028 Season

In [41]:
# Sets Tiers

tiers_dict = tiers(eloranks)
tier1 = tiers_dict["tier1"]
tier2 = tiers_dict["tier2"]
tier3 = tiers_dict["tier3"]
tier4 = tiers_dict["tier4"]

# Runs Season Sim

# tier 1 results
final_results, matchup_df = simulate_season(tier1)
# tier 2 results
final_results2, matchup_df2 = simulate_season(tier2)
# tier 3 results
final_results3, matchup_df3 = simulate_season(tier3)
# tier 4 results
final_results4, matchup_df4 = simulate_season(tier4)


final_results['Tier'] = "Tier 1"
final_results2['Tier'] = "Tier 2"
final_results3['Tier'] = "Tier 3"
final_results4['Tier'] = "Tier 4"

Tier_1_CBR_28 = matchup_df['CBR'].mean()
Tier_2_CBR_28 = matchup_df2['CBR'].mean()
Tier_3_CBR_28 = matchup_df3['CBR'].mean()
Tier_4_CBR_28 = matchup_df4['CBR'].mean()


# Final Season Results

cfb28 = pd.concat([final_results, final_results2, final_results3, final_results4], ignore_index=True)
cfb28['Year'] = 2028
cfb28

Tier_1_CBR_28

17.17328431372549

In [37]:
CBR24 = (Tier_1_CBR_24 + Tier_2_CBR_24 + Tier_3_CBR_24 + Tier_4_CBR_24) / 4
CBR25 = (Tier_1_CBR_25 + Tier_2_CBR_25 + Tier_3_CBR_25 + Tier_4_CBR_25) / 4
CBR26 = (Tier_1_CBR_26 + Tier_2_CBR_26 + Tier_3_CBR_26 + Tier_4_CBR_26) / 4
CBR27 = (Tier_1_CBR_27 + Tier_2_CBR_27 + Tier_3_CBR_27 + Tier_4_CBR_27) / 4
CBR28 = (Tier_1_CBR_28 + Tier_2_CBR_28 + Tier_3_CBR_28 + Tier_4_CBR_28) / 4

CBR5Year = (CBR24 + CBR25 + CBR26 + CBR27 + CBR28) / 5

CBR5Year

14.17256280637255